In [1]:
import sys
sys.path.append("../")
sys.path.append('C:\Users\James Pino\PycharmProjects\Magine')
from IPython.display import display
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
import magine.ontology.enrichment_tools as et
import magine.networks.visualization.notebook_tools as nt
import magine.networks.visualization.notebooks.view as view
from magine.networks.network_subgraphs import NetworkSubgraphs
from exp_data import exp_data

# Exploring network using enrichment analysis

### This example uses the enrichment output we just obtained to explore the network.

First, lets load the molecular network and the enrichment output from the previous runs. 

In [2]:
enrichment_array = pd.read_csv('Data/cisplatin_enrichment.csv.gz', index_col=0)
network = nx.read_gpickle('Data/cisplatin_based_network.p')
subgraph_gen = NetworkSubgraphs(network=network, exp_data=exp_data)

For this example, we will only look at the proteomics data. 

In [3]:
proteomics = et.filter_db(enrichment_array, column='category', options=['proteomics_up'])
proteomics = proteomics[~proteomics['term_name'].isnull()]

In [4]:
print_cols = ['term_name', 'combined_score', 'adj_p_value', 'n_genes', 'rank']

For this example, we just want to look at "biological processes" descriptions, so we will limit our analysis to only databases with this type of information

In [5]:
print(proteomics['db'].unique())

process_dbs = [
        'GO_Biological_Process_2017',
        'Humancyc_2016',
        'Reactome_2016',
        'KEGG_2016',
        'BioCarta_2016',
        'Humancyc_2016',
        'NCI-Nature_2016',
        'Panther_2016',
        'WikiPathways_2016',
]

display(proteomics.head(10))
display(proteomics[print_cols].head(10))
# display(et.filter_based_on_words(proteomics[print_cols+['sample_id']], ['repair']))

['KEGG_2016' 'NCI-Nature_2016' 'Panther_2016' 'WikiPathways_2016'
 'BioCarta_2016' 'Humancyc_2016' 'Reactome_2016' 'KEA_2015'
 'GO_Biological_Process_2017' 'GO_Molecular_Function_2017'
 'GO_Cellular_Component_2017' 'ChEA_2016' 'TRANSFAC_and_JASPAR_PWMs'
 'ENCODE_TF_ChIP-seq_2015' 'Kinase_Perturbations_from_GEO_down'
 'Kinase_Perturbations_from_GEO_up'
 'LINCS_L1000_Kinase_Perturbations_down'
 'LINCS_L1000_Kinase_Perturbations_up'
 'Ligand_Perturbations_from_GEO_down' 'Ligand_Perturbations_from_GEO_up'
 'DrugMatrix' 'Drug_Perturbations_from_GEO_2014']


,term_name,rank,combined_score,adj_p_value,genes,n_genes,sample_id,category,db
188172,Cell cycle_Homo sapiens_hsa04110,1,14.41309,0.05063,"CDC16,CDC20,CDC7,HDAC1,MYC,PLK1,RBL1,SMC1A,YWHAZ",9,01hr,proteomics_up,KEGG_2016
188173,Ubiquitin mediated proteolysis_Homo sapiens_hsa04120,2,12.28734,0.05290,"BRCA1,CDC16,CDC20,CUL3,CUL4A,CUL4B,HUWE1,PIAS1,UBA1",9,01hr,proteomics_up,KEGG_2016
188174,Fanconi anemia pathway_Homo sapiens_hsa03460,3,10.71077,0.13116,"BRCA1,ERCC4,SLX4,TELO2,USP1",5,01hr,proteomics_up,KEGG_2016
188175,Colorectal cancer_Homo sapiens_hsa05210,4,10.51834,0.18012,"APC,APPL1,BRAF,MAPK3,MYC",5,01hr,proteomics_up,KEGG_2016
188176,Longevity regulating pathway - multiple species_Homo sapiens_hsa04213,5,10.14721,0.18012,"EIF4EBP2,HDAC1,HSPA2,INSR,IRS2",5,01hr,proteomics_up,KEGG_2016
188177,Chronic myeloid leukemia_Homo sapiens_hsa05220,6,8.49857,0.20252,"BRAF,HDAC1,MAPK3,MYC,SHC1",5,01hr,proteomics_up,KEGG_2016
188178,Spliceosome_Homo sapiens_hsa03040,7,8.19675,0.20252,"DDX5,HNRNPA1,HNRNPM,HSPA2,PRPF3,SRSF9,THOC2",7,01hr,proteomics_up,KEGG_2016
188179,Nucleotide excision repair_Homo sapiens_hsa03420,8,7.57820,0.20252,"CUL4A,CUL4B,ERCC4,ERCC5",4,01hr,proteomics_up,KEGG_2016
188180,Endometrial cancer_Homo sapiens_hsa05213,9,7.47564,0.25385,"APC,BRAF,MAPK3,MYC",4,01hr,proteomics_up,KEGG_2016
188181,Oocyte meiosis_Homo sapiens_hsa04114,10,6.43965,0.29695,"CDC16,CDC20,MAPK3,PLK1,SMC1A,YWHAZ",6,01hr,proteomics_up,KEGG_2016


,term_name,combined_score,adj_p_value,n_genes,rank
188172,Cell cycle_Homo sapiens_hsa04110,14.41309,0.05063,9,1
188173,Ubiquitin mediated proteolysis_Homo sapiens_hsa04120,12.28734,0.05290,9,2
188174,Fanconi anemia pathway_Homo sapiens_hsa03460,10.71077,0.13116,5,3
188175,Colorectal cancer_Homo sapiens_hsa05210,10.51834,0.18012,5,4
188176,Longevity regulating pathway - multiple species_Homo sapiens_hsa04213,10.14721,0.18012,5,5
188177,Chronic myeloid leukemia_Homo sapiens_hsa05220,8.49857,0.20252,5,6
188178,Spliceosome_Homo sapiens_hsa03040,8.19675,0.20252,7,7
188179,Nucleotide excision repair_Homo sapiens_hsa03420,7.57820,0.20252,4,8
188180,Endometrial cancer_Homo sapiens_hsa05213,7.47564,0.25385,4,9
188181,Oocyte meiosis_Homo sapiens_hsa04114,6.43965,0.29695,6,10


In [6]:
time_1_hour_prot = et.filter_dataframe(proteomics, 
                                       p_value=0.05,
                                       combined_score=0.0, 
                                       sample_id='01hr', 
                                       db=process_dbs,
                                      )
display(time_1_hour_prot[print_cols].sort_values('rank').head(15))

,term_name,combined_score,adj_p_value,n_genes,rank
189887,"negative regulation of transcription, DNA-templated",68.26198,0.00160,22,1
189000,Cell Cycle_Homo sapiens_R-HSA-1640170,36.50732,0.00027,28,1
188379,Validated targets of C-MYC transcriptional repression_Homo sapiens_6bbdafa6-6196-11e5-8ac5-06603...,15.36952,0.00957,7,1
188581,XPodNet - protein-protein interactions in the podocyte expanded by STRING_Mus musculus_WP2309,22.46025,0.01109,30,1
188380,Regulation of Telomerase_Homo sapiens_4dfe97ca-6195-11e5-8ac5-06603eb7f303,14.71910,0.00957,7,2
188582,Retinoblastoma (RB) in Cancer_Homo sapiens_WP2446,17.16690,0.01763,8,2
189001,Interleukin-2 signaling_Homo sapiens_R-HSA-451927,30.22085,0.00159,16,2
189002,"Interleukin-3, 5 and GM-CSF signaling_Homo sapiens_R-HSA-512988",29.30395,0.00159,16,3
189889,resolution of meiotic recombination intermediates,29.55728,0.01649,4,3
188583,EGFR1 Signaling Pathway_Mus musculus_WP572,15.61893,0.04176,10,3


Since terms across databases might be redundant ("Interleukin-3, 5 and GM-CSF signaling_Homo sapiens_R-HSA-512988" and "Interleukin receptor SHC signaling_Homo sapiens_R-HSA-912526" have nearly a full overlap of genes), we want to eliminate duplicate terms and focus on the most enriched. For that we use the Jaccard Index. It is implmented in the find_similar_terms function in enrichment_tools (et).

Now we can explore the top hits, which has been slimmed from 89 to 33 terms. Generally this is where the expert knowledge comes in. However, a quick search with each term and search terms of you molecule of interest tend to be useful. 

The first hit is 'negative regulation of transcription', which means that something caused genes not to be transcribed. Cisplatin causes DNA damage, thus negative regulaton of transcription makes sense. So does top hit 2, 'Cell Cycle_Homo sapiens_R-HSA-1640170'. 

A quick search for 'Interleukin-2 signaling' and 'Cisplatin' __[link](https://www.google.com/search?rlz=1C1CHBD_enUS721US721&ei=KzNeWuCxBsfq_AaSgYuYDQ&q=Interleukin-2+signaling+cisplatin&oq=Interleukin-2+signaling+cisplatin&gs_l=psy-ab.3..35i39k1.8097.9052.0.9196.10.10.0.0.0.0.145.897.7j3.10.0....0...1c.1.64.psy-ab..3.2.218....0.TInUjcZY740)__ returns a paper titled "Cisplatin at clinically relevant concentrations enhances interleukin-2 synthesis by human primary blood lymphocytes." __[link](https://www.ncbi.nlm.nih.gov/pubmed/10211553)__


We can link the two together to see how once might regulate the other by looking at the molecular interactions.

## Expected findings

In [7]:
filtered_1hr = et.filter_similar_terms(time_1_hour_prot, threshold=.7)
display(filtered_1hr[print_cols].head(25))

Number of rows went from 89 to 25


,term_name,combined_score,adj_p_value,n_genes,rank
189887,"negative regulation of transcription, DNA-templated",68.26198,0.00160,22,1
189000,Cell Cycle_Homo sapiens_R-HSA-1640170,36.50732,0.00027,28,1
189001,Interleukin-2 signaling_Homo sapiens_R-HSA-451927,30.22085,0.00159,16,2
189889,resolution of meiotic recombination intermediates,29.55728,0.01649,4,3
189008,Signalling by NGF_Homo sapiens_R-HSA-166520,25.78307,0.00159,21,9
189893,mitotic cytokinesis,24.79824,0.03095,5,7
188581,XPodNet - protein-protein interactions in the podocyte expanded by STRING_Mus musculus_WP2309,22.46025,0.01109,30,1
189894,chloride transport,22.31045,0.02926,5,8
189032,Axon guidance_Homo sapiens_R-HSA-422475,20.50960,0.00159,22,33
188582,Retinoblastoma (RB) in Cancer_Homo sapiens_WP2446,17.16690,0.01763,8,2


In [8]:
shorten_names = {
    'Cell Cycle_Homo sapiens_R-HSA-1640170':'Cell Cycle',
    'negative regulation of transcription, DNA-templated': 'Anti-transcription',
    'DNA Repair_Homo sapiens_R-HSA-73894' : 'DNA Repair',
#     'Interleukin-2 signaling_Homo sapiens_R-HSA-451927': 'IL2',
#     'resolution of meiotic recombination intermediates': 'Meiotic Recombination', 
#     'E2F transcription factor network_Homo sapiens_bb4d0fd3-6191-11e5-8ac5-06603eb7f303' : 'E2F tfs'
                }
renamed_1hr = filtered_1hr.copy()
renamed_1hr['term_name'] = renamed_1hr['term_name'].replace(shorten_names)
term_net_1, mol_net_1 = nt.create_subnetwork(shorten_names.values(), renamed_1hr, network, '1hr')


('DNA Repair', 12)
('Cell Cycle', 28)
('Anti-transcription', 22)
Looking for direct edges


In [9]:
view.display_graph(term_net_1)

In [10]:
view.display_graph(mol_net_1, add_parent=True)

In [11]:
print(len(mol_net_1.nodes()))
print(len(mol_net_1.edges()))
plus_neighbors = subgraph_gen.expand_neighbors(mol_net_1, nodes=[], down_stream=True, expand_all=True, include_list=exp_data.sig_species_over_time['06hr'])
print(len(plus_neighbors.nodes()))
print(len(plus_neighbors.edges()))

# subgraph_gen.measured_networks_over_time(plus_neighbors, ['red', 'green', 'lightblue', 'purple'], 'first_time_point')

12
11
Removing from list
163
194


In [14]:
view.render_graph(plus_neighbors)

In [13]:
cell_cycle = et.term_to_genes(df=renamed_1hr, term='Cell Cycle')
print(len(cell_cycle))

cell_cycle = subgraph_gen.shortest_paths_between_lists(
    cell_cycle, max_length=2, pool=None
)
print(len(cell_cycle.nodes()))
print(len(cell_cycle.edges()))

28
Removing from list
22
23


In [ ]:
# view.render_graph(negative_reg_transcription_network)
subgraph_gen.measured_networks_over_time(mol_net_1, ['red', 'green', 'lightblue', 'purple'], 'transcription')

In [ ]:
# dna_genes = et.term_to_genes(df=proteomics, term='DNA Repair_Homo sapiens_R-HSA-73894')
dna_genes = et.term_to_genes(df=renamed_1hr, term='DNA Repair')

In [ ]:
dna_network = subgraph_gen.shortest_paths_between_lists(
    dna_genes, max_length=3
)

In [ ]:
print(len(dna_network.nodes()))

In [ ]:
view.render_graph(dna_network)

In [ ]:
subgraph_gen.measured_networks_over_time(dna_network, ['red', 'green', 'lightblue', 'purple'], 'dna')

In [ ]:
from magine.plotting.species_plotting import plot_list_of_genes
plot_list_of_genes(exp_data.data, dna_genes, save_name='dna_genes', image_format='png', plot_type='matplotlib')

In [ ]:
time_1_hour_prot = et.filter_dataframe(proteomics, 
                                       p_value=0.05,
                                       combined_score=0.0, 
                                       sample_id='01hr', 
                                       db=process_dbs,
                                      )
kinases = et.filter_dataframe(proteomics, 
                              p_value=0.05, 
                              sample_id='01hr', 
                              combined_score=0.0, 
                              db='KEA_2015')
kinases = et.remove_redundant(kinases)

display(kinases[print_cols].head(20))

## Exploring other top hits from 1 hour

### Side effects of cisplatin
Chemotherapy-induced peripheral neuropathy. __[link](https://www.frontiersin.org/articles/10.3389/fnins.2017.00481/full)__
It is not well understood why cisplatin causes CIPN. Surprisely here, we see that Axon Guidance has a combined score of 20.5. 33 species are effected by cisplain that are linked with axon guidance. We are not neural experts and did not know that axon guidance was related to CIPN. Using MAGINE we were able to find ties between the two.

In [ ]:
axon_guidance = et.term_to_genes(df=renamed_1hr, term='Axon guidance_Homo sapiens_R-HSA-422475')
print(axon_guidance)

In [ ]:
g = subgraph_gen.neighbors_of_list(axon_guidance, up_stream=True, down_stream=False, max_dist=1, 
                                   include_only=exp_data.sig_species_over_time['01hr']
                                  )
nt.render_graph(g)

In [ ]:
chloride_transport = et.term_to_genes(df=renamed_1hr, term='chloride transport')
print(chloride_transport)
chl_trans = subgraph_gen.neighbors_of_list(chloride_transport, max_dist=2, 
#                                            include_only=exp_data.list_species
                                          )
nt.display_graph(chl_trans)

# 6 hour time point

In [ ]:
time_6_hour_prot = et.filter_dataframe(proteomics, 
                                       p_value=0.05, 
                                       combined_score=0.0, 
                                       sample_id='06hr', 
                                       db=process_dbs
                                      )
slimmed = et.filter_similar_terms(time_6_hour_prot, threshold=.7)
display(slimmed[print_cols].head(25))

In [ ]:
shorten_names = {
    'protein sumoylation':'protein sumoylation',
    'Activation of the AP-1 family of transcription factors_Homo sapiens_R-HSA-450341' : 'AP1 activation',
    'response to cAMP' : 'response to cAMP',
                }
time_6_hour_prot['term_name'] = time_6_hour_prot['term_name'].replace(shorten_names)
term_net_6, mol_net_6 = nt.create_subnetwork(shorten_names.values(),
                                             time_6_hour_prot, 
                                             network,
                                             '06hr')


In [ ]:
view.display_graph(term_net_6)

In [ ]:
view.render_graph(mol_net_6)

In [ ]:
# nt.find_neighbors(g=mol_net_1, start='ATR', up_stream=False, down_stream=True, max_dist=1, render=True)
# nt.find_neighbors(g=mol_net_1, start='ATR', up_stream=True, down_stream=False, max_dist=3, render=True)

In [ ]:
time_24_hour_prot = et.filter_dataframe(proteomics, p_value=0.05, combined_score=0.0, category='proteomics_up', sample_id='24hr', db=process_dbs)
slimmed = et.filter_similar_terms(time_24_hour_prot, threshold=.5)
display(slimmed[print_cols].head(25))

In [ ]:
shorten_names = {
                 'cellular response to DNA damage stimulus': 'DDR',
                 'negative regulation of apoptotic process': 'negative regulation of apoptosis',
                 'Apoptosis_Homo sapiens_R-HSA-109581' : 'Apoptosis',
                }
renamed = time_24_hour_prot.copy()
time_24_hour_prot['term_name'] = time_24_hour_prot['term_name'].replace(shorten_names)
term_net_24, mol_net_24 = nt.create_subnetwork(shorten_names.values(), time_24_hour_prot, network, '24hr', cytoscape_js=False)

In [ ]:
view.display_graph(term_net_24)

In [ ]:
view.display_graph(mol_net_24, add_parent=True )

In [ ]:
time_48_hour_prot = et.filter_dataframe(proteomics, p_value=0.05, combined_score=0.0, category='proteomics_up', sample_id='48hr', db=process_dbs)
slimmed = et.filter_similar_terms(time_48_hour_prot, threshold=.5)
display(slimmed[print_cols].head(25))

In [ ]:
shorten_names = {
                 'membrane organization': 'Membrane Organization',
                 'negative regulation of apoptotic process': 'negative regulation of apoptosis',
                 'neutrophil degranulation' : 'neutrophil degranulation',
                }
time_48_hour_prot['term_name'] = time_48_hour_prot['term_name'].replace(shorten_names)
term_net_48, mol_net_48 = nt.create_subnetwork(shorten_names.values(), time_48_hour_prot, network, '48hr')

In [ ]:
view.display_graph(term_net_48)

In [ ]:
view.render_graph(mol_net_48, add_parent=True)